# COBRApy

COBRApy is a package for constraint-based modeling of biological networks written in Python.

This tool allows loading and inspecting Genome-Scale Metabolic (GEM) models written in the Systems Biology Markup Language (SBML) format.

Using COBRApy, one can analyze the following model contents:
1. Reactions
2. Metabolites
3. Genes
4. Exchange reactions (Environmental Conditions)

COBRApy allows manipulating the contents of a GEM model. For instance, one can edit reactions' flux bounds, knock out a metabolic gene, or change environmental conditions.

COBRApy contains flux analysis methods to simulate an organism's phenotypic behavior. These include Flux Balance Analysis (FBA), Parsimonious FBA, or Flux Variability Analysis (FVA).

The simulation of gene and reaction deletions for a given GEM model is a straightforward process. One can simulate single or double knockouts using one of the flux analysis methods mentioned above.

## Installation


### Requirements
The following requirements are needed to use COBRApy:
- Python 3.6 or higher
- pip must be installed
- GLPK is the default solver, but CPLEX is preferred


### How to install COBRApy?
```
pip install cobra
```

# Exercise 7 - Phenotype prediction

## Working with a GEM model

For this practical session, we will be using the following model:
- _E. coli_ core model which contains the central carbon metabolism of _Escherichia coli_ -> file: ../data/e_coli_core.xml

You can read more about _E. coli_ core model (Orth et al., 2010) in the following links:
- https://journals.asm.org/doi/10.1128/ecosalplus.10.2.1
- http://bigg.ucsd.edu/models/e_coli_core

This exercise consists of exploring the phenotype prediction tools of COBRApy. Thus, the following steps will be followed:
- Perform reaction and gene deletions;
- Perform mutant-specific simulation methods, such as ROOM and MOMA;
- Perform single and double deletions

In [1]:
# imports
import cobra
import escher
# Loading a model
model_path = '../data/e_coli_core.xml'
model = cobra.io.read_sbml_model(model_path)

model

## Phenotype Prediction

COBRApy includes different algorithms for phenotype prediction. These include:
- Flux Balance Analysis (FBA); 
- Parsimonious Flux Balance Analysis (pFBA);
- Flux Variability Analysis (FVA);
- Regulatory On/Off Minimization (ROOM);
- Minimization of Metabolic Adjustment (MOMA);

### Simulating Deletions

As previously mentioned, COBRApy can be used to simulate gene or reaction deletions. The function `knock_out()` can be used to knock out a given reaction or gene.

In [2]:
#knock out the SUCDi reaction
with model:
    model.reactions.SUCDi.knock_out()
    pfba_solution = cobra.flux_analysis.pfba(model)
    print('SUCDi mutant growth rate: ', pfba_solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])
    print('SUCDi flux rate: ', pfba_solution.fluxes['SUCDi'])
    print('SUCDi mutant succinate production rate: ', pfba_solution.fluxes['EX_succ_e'])

In [3]:
# knock out the b1852 gene associated with reaction G6PDH2r
with model:
    model.genes.b1852.knock_out()
    pfba_solution = cobra.flux_analysis.pfba(model)
    print('b1852 mutant growth rate: ', pfba_solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])
    print('G6PDH2r flux rate: ', pfba_solution.fluxes['G6PDH2r'])
    print('b1852 mutant succinate production rate: ', pfba_solution.fluxes['EX_succ_e'])

Gene-Protein-Reaction (GPR) rules can be used to understand which genes are associated with a given reaction. Besides, one can understand by the GPR rule if the reaction is being catalyzed by a single gene, isoenzyme or protein complex. In COBRApy, one can inspect the GPR rule of a given reaction or which reactions are associated with a given gene.

In [4]:
# ACKr GPR rule. This reaction is being catalyzed by an isoenzyme
model.reactions.ACKr.gene_reaction_rule

In [5]:
# knock out the b3115 & b2296 & b1849 genes associated with reaction ACKr
with model:
    model.genes.b3115.knock_out()
    model.genes.b2296.knock_out()
    model.genes.b1849.knock_out()
    pfba_solution = cobra.flux_analysis.pfba(model)
    print('b3115, b2296, b1849 mutant growth rate: ', pfba_solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])
    print('ACKr flux rate: ', pfba_solution.fluxes['ACKr'])
    print('b3115, b2296, b1849 mutant succinate production rate: ', pfba_solution.fluxes['EX_succ_e'])

In [6]:
# performing all deletions at once to verify succinate production rate, EX_succ_e
with model:
    wt_pfba_solution = cobra.flux_analysis.pfba(model)
    print('WT growth rate: ', wt_pfba_solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])
    print('WT succinate production rate: ', wt_pfba_solution.fluxes['EX_succ_e'])
    model.reactions.SUCDi.knock_out()
    model.genes.b1852.knock_out()
    model.genes.b3115.knock_out()
    model.genes.b2296.knock_out()
    model.genes.b1849.knock_out()
    mutant_pfba_solution = cobra.flux_analysis.pfba(model)
    print('MUTANT growth rate: ', mutant_pfba_solution.fluxes['BIOMASS_Ecoli_core_w_GAM'])
    print('MUTANT succinate production rate: ', mutant_pfba_solution.fluxes['EX_succ_e'])

In [7]:
# WILD-TYPE flux distribution
builder = escher.Builder(map_name='e_coli_core.Core metabolism', model=model, reaction_data=wt_pfba_solution.fluxes)
builder

In [8]:
# MUTANT flux distribution
builder = escher.Builder(map_name='e_coli_core.Core metabolism', model=model, reaction_data=mutant_pfba_solution.fluxes)
builder

### MOMA and ROOM

COBRApy includes phenotype prediction methods that are used predict the flux distribution after a gene knock out. These are the Minimization of Metabolic Adjustment (MOMA), which can be called using `cobra.flux_analysis.moma()`, and Regulatory On/Off Minimization (ROOM), using `cobra.flux_analysis.room()`.

In [9]:
#using MOMA with COBRApy
with model:
    pfba_solution = cobra.flux_analysis.pfba(model)
    model.reactions.SUCDi.knock_out()
    model.genes.b1852.knock_out()
    model.genes.b3115.knock_out()
    model.genes.b2296.knock_out()
    model.genes.b1849.knock_out()
    moma_result = cobra.flux_analysis.moma(model, pfba_solution)
    print('MOMA Result: ', moma_result)

In [10]:
model.summary(moma_result)

In [11]:
#using ROOM with COBRApy
with model:
    pfba_solution = cobra.flux_analysis.pfba(model)
    model.reactions.SUCDi.knock_out()
    model.genes.b1852.knock_out()
    model.genes.b3115.knock_out()
    model.genes.b2296.knock_out()
    model.genes.b1849.knock_out()
    room_result = cobra.flux_analysis.room(model, pfba_solution)
    print('ROOM Result: ', room_result)

In [12]:
model.summary(room_result)

### Single Deletions

Single gene and reaction deletions can also be simulated with the flux analysis package of COBRApy. To do so, one can use the `cobra.flux_analysis.single_gene_deletion()` and `cobra.flux_analysis.single_reaction_deletion()` methods.

In [13]:
#single reaction deletion
reaction_deletion_results = cobra.flux_analysis.single_reaction_deletion(model)
reaction_deletion_results

In [14]:
#single gene deletion
gene_deletion_results = cobra.flux_analysis.single_gene_deletion(model)
gene_deletion_results

It is worth noting that genes and reactions with a growth rate equal to zero can be considered as essential genes or essential reactions, respectively.

### Double Deletions

Double gene and reaction deletions can also be simulated with the flux analysis package of COBRApy. To do so, one can use the `cobra.flux_analysis.double_gene_deletion()` and `cobra.flux_analysis.double_reaction_deletion()` methods. These methods perform all possible combinations.

In [15]:
#double reaction deletion
double_reaction_deletion_results = cobra.flux_analysis.double_reaction_deletion(model)
double_reaction_deletion_results

In [16]:
#double gene deletion
double_gene_deletion_results = cobra.flux_analysis.double_gene_deletion(model)
double_gene_deletion_results